In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR_AKB = "/content/drive/MyDrive/Colab Notebooks/akimoto/gan_saka"


In [3]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--n_epoch", type=int, default=200)
parser.add_argument("--batch_size", type=int, default=64)
parser.add_argument("--lr", type=float, default=2e-4)
parser.add_argument("--nch_g", type=int, default=128)
parser.add_argument("--nch_d", type=int, default=128)
parser.add_argument("--z_dim", type=int, default=100) # noize dimension
parser.add_argument("--beta1", type=float, default=0.5) # for Adam optimizer
opt = parser.parse_args(args=[])
print(opt)
print(opt.lr)

Namespace(batch_size=64, beta1=0.5, lr=0.0002, n_epoch=200, nch_d=128, nch_g=128, z_dim=100)
0.0002


In [5]:
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, file_list, transform=None):
        super().__init__()
        self.file_list = file_list
        self.transform = transform
    
    def __getitem__(self, index):
        # If transform exists, do pre_process
        if self.transform is not None:
            img = self.transform(Image.open(self.file_list[index]))
        else:
            img = Image.open(self.file_list[index])
        return img

    def __len__(self):
        return len(self.file_list)


In [6]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

In [7]:
import torchvision.datasets as dset

In [8]:
dataset = dset.MNIST("./", train=True, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [9]:
dataset = dset.MNIST("/content/drive/MyDrive/data", download=False, train=True, transform=transform)

In [10]:
from torch.utils.data import DataLoader

In [11]:
dataloader = DataLoader(dataset=dataset, batch_size=opt.batch_size,
                        shuffle=True)

In [12]:
import torch.nn as nn

In [13]:
def weights_init(m):
    # m means model
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)   # initialize using normal distribution
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)   # initialize using normal distribution
        nn.init.constant_(m.bias.data, 0)   # initialize using normal distribution
